# Newton’s Method via JAX

```{include} _admonition/gpu.md
```

## Overview

In this lecture we highlight some of the capabilities of JAX, including JIT
compilation and automatic differentiation.

The application is computing equilibria via Newton's method, which we discussed 
in [a more elementary QuantEcon lecture](https://python.quantecon.org/newton_method.html)

Here our focus is on how to apply JAX to this problem.

We use the following imports in this lecture

In [1]:
import jax
import jax.numpy as jnp
from scipy.optimize import root

Let's check the GPU we are running

In [2]:
!nvidia-smi

Tue May 16 11:16:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|


|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   30C    P0    38W / 300W |      0MiB / 16160MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+


## The Equilibrium Problem

In this section we describe the market equilibrium problem we will solve with
JAX.

We begin with a two good case, 
which is borrowed from [an earlier lecture](https://python.quantecon.org/newton_method.html).

Then we shift to higher dimensions.


### The Two Goods Market Equilibrium

Assume we have a market for two complementary goods where demand depends on the
price of both components.

We label them good 0 and good 1, with price vector $p = (p_0, p_1)$.

Then the supply of good $i$ at price $p$ is,

$$
q^s_i (p) = b_i \sqrt{p_i}
$$

and the demand of good $i$ at price $p$ is,

$$
q^d_i (p) = \text{exp}(-(a_{i0} p_0 + a_{i1} p_1)) + c_i
$$

Here $a_{ij}$, $b_i$ and $c_i$ are parameters for $n \times n$ square matrix $A$ and $n \times 1$ parameter vectors $b$ and $c$.

The excess demand function is,

$$
e_i(p) = q_i^d(p) - q_i^s(p), \quad i = 0, 1
$$

An equilibrium price vector $p^*$ satisfies $e_i(p^*) = 0$.

We set

$$
A = \begin{pmatrix}
            a_{00} & a_{01} \\
            a_{10} & a_{11}
        \end{pmatrix},
            \qquad
    b = \begin{pmatrix}
            b_0 \\
            b_1
        \end{pmatrix}
    \qquad \text{and} \qquad
    c = \begin{pmatrix}
            c_0 \\
            c_1
        \end{pmatrix}
$$

for this particular question.


### A High-Dimensional Version

Let's now shift to a linear algebra formulation, which alllows us to handle
arbitrarily many goods.

The supply function remains unchanged,

$$
    q^s (p) =b \sqrt{p}
$$

The demand function becomes

$$
    q^d (p) = \text{exp}(- A \cdot p) + c
$$

Our new excess demand function is

$$
e(p) = \text{exp}(- A \cdot p) + c - b \sqrt{p}
$$

The function below calculates the excess demand for the given parameters

In [3]:
def e(p, A, b, c):
    return jnp.exp(- A @ p) + c - b * jnp.sqrt(p)

## Computation

In this section we describe and then implement the solution method.


### Newton's Method

We use a multivariate version of Newton's method to compute the equilibrium price.

The rule for updating a guess $p_n$ of the price vector is

```{math}
:label: multi-newton
p_{n+1} = p_n - J_e(p_n)^{-1} e(p_n)
```

Here $J_e(p_n)$ is the Jacobian of $e$ evaluated at $p_n$.

Iteration starts from initial guess $p_0$.

Instead of coding the Jacobian by hand, we use `jax.jacobian()`.

In [4]:
def newton(f, x_0, tol=1e-5, max_iter=15):
    x = x_0
    f_jac = jax.jacobian(f)
    q = jax.jit(lambda x: x - jnp.linalg.solve(f_jac(x), f(x)))
    error = tol + 1
    n = 0
    while error > tol:
        n += 1
        if(n > max_iter):
            raise Exception('Max iteration reached without convergence')
        y = q(x)
        if jnp.any(jnp.isnan(y)):
            raise Exception('Solution not found with NaN generated')
        error = jnp.linalg.norm(x - y)
        x = y
        print(f'iteration {n}, error = {error}')
    print('\n' + f'Result = {x} \n')
    return x

### Application

Let's now apply the method just described to investigate a large market with 5,000 goods.

We randomly generate the matrix $A$ and set the parameter vectors $b \text{ and } c$ to $1$.

In [5]:
dim = 5_000
seed = 32

# Create a random matrix A and normalize the rows to sum to one
key = jax.random.PRNGKey(seed)

A = jax.random.uniform(key, [dim, dim])

s = jnp.sum(A, axis=0)
A = A / s

# Set up b and c
b = jnp.ones(dim)
c = jnp.ones(dim)

Here's our initial condition $p_0$

In [6]:
init_p = jnp.ones(dim)

By leveraging the power of Newton's method, JAX accelerated linear algebra,
automatic differentiation, and a GPU, we obtain a relatively small error for
this very large problem in just a few seconds:

In [7]:
%%time

p = newton(lambda p: e(p, A, b, c), init_p).block_until_ready()

iteration 1, error = 29.97745704650879
iteration 2, error = 5.092828750610352
iteration 3, error = 0.10971643775701523
iteration 4, error = 5.197196878725663e-05


iteration 5, error = 1.2309188605286181e-05
iteration 6, error = 4.6261807256087195e-06

Result = [1.4999796 1.503175  1.4918782 ... 1.4914232 1.4956646 1.4976945] 

CPU times: user 4.76 s, sys: 1.44 s, total: 6.21 s
Wall time: 4.26 s


In [8]:
jnp.max(jnp.abs(e(p, A, b, c)))

Array(1.1920929e-07, dtype=float32)

With the same tolerance, SciPy's `root` function takes much longer to run,
even with the Jacobian supplied.

In [9]:
%%time

solution = root(lambda p: e(p, A, b, c),
                init_p,
                jac=lambda p: jax.jacobian(e)(p, A, b, c),
                method='hybr',
                tol=1e-5)

CPU times: user 2min 25s, sys: 381 ms, total: 2min 26s
Wall time: 2min 26s


In [10]:
p = solution.x
jnp.max(jnp.abs(e(p, A, b, c)))

Array(7.1525574e-07, dtype=float32)

The result is also less accurate.



## Exercises

```{exercise-start}
:label: newton_ex1
```

Consider a three-dimensional extension of [the Solow fixed point
problem](https://python.quantecon.org/newton_method.html#the-solow-model) with

$$
A = \begin{pmatrix}
            2 & 3 & 3 \\
            2 & 4 & 2 \\
            1 & 5 & 1 \\
        \end{pmatrix},
            \quad
s = 0.2, \quad α = 0.5, \quad δ = 0.8
$$

As before the law of motion is

```{math}
    k_{t+1} = g(k_t) \quad \text{where} \quad
    g(k) := sAk^\alpha + (1-\delta) k
```

However $k_t$ is now a $3 \times 1$ vector.

Solve for the fixed point using Newton's method with the following initial values:

$$
\begin{aligned}
    k1_{0} &= (1, 1, 1) \\
    k2_{0} &= (3, 5, 5) \\
    k3_{0} &= (50, 50, 50)
\end{aligned}
$$

````{hint}
:class: dropdown
- The computation of the fixed point is equivalent to computing $k^*$ such that $f(k^*) - k^* = 0$.
- If you are unsure about your solution, you can start with the solved example:
```{math}
A = \begin{pmatrix}
            2 & 0 & 0 \\
            0 & 2 & 0 \\
            0 & 0 & 2 \\
        \end{pmatrix}
```
with $s = 0.3$, $α = 0.3$, and $δ = 0.4$ and starting value:
```{math}
k_0 = (1, 1, 1)
```
The result should converge to the [analytical solution](https://python.quantecon.org/newton_method.html#solved-k).
````

```{exercise-end}
```


```{solution-start} newton_ex1
:class: dropdown
```

Let's first define the parameters for this problem

In [11]:
A = jnp.array([[2.0, 3.0, 3.0],
               [2.0, 4.0, 2.0],
               [1.0, 5.0, 1.0]])
s = 0.2
α = 0.5
δ = 0.8
initLs = [jnp.ones(3),
          jnp.array([3.0, 5.0, 5.0]),
          jnp.repeat(50.0, 3)]

Then define the multivariate version of the formula for the [law of motion of captial](https://python.quantecon.org/newton_method.html#solow)

In [12]:
def multivariate_solow(k, A=A, s=s, α=α, δ=δ):
    return s * jnp.dot(A, k**α) + (1 - δ) * k

Let's run through each starting value and see the output

In [13]:
attempt = 1
for init in initLs:
    print(f'Attempt {attempt}: Starting value is {init} \n')
    %time k = newton(lambda k: multivariate_solow(k) - k, \
                     init).block_until_ready()
    print('-'*64)
    attempt += 1

Attempt 1: Starting value is [1. 1. 1.] 



iteration 1, error = 50.496315002441406
iteration 2, error = 41.1093864440918
iteration 3, error = 4.294127464294434
iteration 4, error = 0.3854290544986725
iteration 5, error = 0.0054382034577429295
iteration 6, error = 8.92080606718082e-07

Result = [3.8405814 3.870718  3.4109194] 

CPU times: user 805 ms, sys: 25.2 ms, total: 830 ms
Wall time: 678 ms
----------------------------------------------------------------
Attempt 2: Starting value is [3. 5. 5.] 



iteration 1, error = 2.0701100826263428
iteration 2, error = 0.12642373144626617
iteration 3, error = 0.0006017307168804109
iteration 4, error = 3.3717478231665154e-07

Result = [3.8405814 3.8707182 3.4109197] 

CPU times: user 588 ms, sys: 4.07 ms, total: 592 ms
Wall time: 220 ms
----------------------------------------------------------------
Attempt 3: Starting value is [50. 50. 50.] 



iteration 1, error = 73.00942993164062


iteration 2, error = 6.493789196014404
iteration 3, error = 0.6806989312171936
iteration 4, error = 0.016202213242650032
iteration 5, error = 1.0600916539260652e-05
iteration 6, error = 9.830249609876773e-07

Result = [3.840581  3.8707182 3.41092  ] 

CPU times: user 1.23 s, sys: 3.97 ms, total: 1.23 s
Wall time: 508 ms
----------------------------------------------------------------


We find that the results are invariant to the starting values given the well-defined property of this question.

But the number of iterations it takes to converge is dependent on the starting values.

Let substitute the output back to the formulate to check our last result

In [14]:
multivariate_solow(k) - k

Array([ 4.7683716e-07,  0.0000000e+00, -2.3841858e-07], dtype=float32)

Note the error is very small.

We can also test our results on the known solution

In [15]:
A = jnp.array([[2.0, 0.0, 0.0],
               [0.0, 2.0, 0.0],
               [0.0, 0.0, 2.0]])
s = 0.3
α = 0.3
δ = 0.4
init = jnp.repeat(1.0, 3)
%time k = newton(lambda k: multivariate_solow(k, A=A, s=s, α=α, δ=δ) - k, \
                 init).block_until_ready()

iteration 1, error = 1.5745922327041626


iteration 2, error = 0.21344946324825287
iteration 3, error = 0.002045975998044014
iteration 4, error = 8.259061701210157e-07

Result = [1.7846744 1.7846744 1.7846744] 

CPU times: user 1.16 s, sys: 16.2 ms, total: 1.17 s
Wall time: 507 ms


The result is very close to the ground truth but still slightly different.

We can increase the precision of the floating point numbers and restrict the tolerance to obtain a more accurate approximation (see detailed discussion in the [lecture on JAX](https://python-programming.quantecon.org/jax_intro.html#differences))

In [16]:
# We will use 64 bit floats with JAX in order to increase the precision.
jax.config.update("jax_enable_x64", True)
init = init.astype('float64')

%time k = newton(lambda k: multivariate_solow(k, A=A, s=s, α=α, δ=δ) - k,\
                 init,\
                 tol=1e-7).block_until_ready()

iteration 1, error = 1.5745916432444333
iteration 2, error = 0.21344933091258958
iteration 3, error = 0.0020465547718452695
iteration 4, error = 2.0309190076799282e-07
iteration 5, error = 1.538370149106851e-15

Result = [1.78467418 1.78467418 1.78467418] 

CPU times: user 848 ms, sys: 16.4 ms, total: 865 ms
Wall time: 697 ms


We can see it steps towards a more accurate solution.

```{solution-end}
```


```{exercise-start}
:label: newton_ex2
```

In this exercise, let's try different initial values and check how Newton's method responds to different starting points.

Let's define a three-good problem with the following default values:

$$
A = \begin{pmatrix}
            0.2 & 0.1 & 0.7 \\
            0.3 & 0.2 & 0.5 \\
            0.1 & 0.8 & 0.1 \\
        \end{pmatrix},
            \qquad
b = \begin{pmatrix}
            1 \\
            1 \\
            1
        \end{pmatrix}
    \qquad \text{and} \qquad
c = \begin{pmatrix}
            1 \\
            1 \\
            1
        \end{pmatrix}
$$

For this exercise, use the following extreme price vectors as initial values:

$$

\begin{aligned}
    p1_{0} &= (5, 5, 5) \\
    p2_{0} &= (1, 1, 1) \\
    p3_{0} &= (4.5, 0.1, 4)
\end{aligned}
$$

Set the tolerance to $10^{-15}$ for more accurate output.


```{hint}
:class: dropdown
Similar to [exercise 1](newton_ex1), enabling `float64` for JAX can improve the precision of our results.
```


```{exercise-end}
```

```{solution-start} newton_ex2
:class: dropdown
```

Define parameters and initial values

In [17]:
A = jnp.array([
    [0.2, 0.1, 0.7],
    [0.3, 0.2, 0.5],
    [0.1, 0.8, 0.1]
])
b = jnp.array([1.0, 1.0, 1.0])
c = jnp.array([1.0, 1.0, 1.0])
initLs = [jnp.repeat(5.0, 3),
          jnp.array([4.5, 0.1, 4.0])]

Let’s run through each initial guess and check the output

In [18]:
attempt = 1
for init in initLs:
    print(f'Attempt {attempt}: Starting value is {init} \n')
    init = init.astype('float64')
    %time p = newton(lambda p: e(p, A, b, c), \
                 init, \
                 tol=1e-15, max_iter=15).block_until_ready()
    print('-'*64)
    attempt +=1

Attempt 1: Starting value is [5. 5. 5.] 



iteration 1, error = 9.243805733085065


Exception: Solution not found with NaN generated

----------------------------------------------------------------
Attempt 2: Starting value is [4.5 0.1 4. ] 

iteration 1, error = 4.892018895185869
iteration 2, error = 1.2120550201694784


iteration 3, error = 0.6942087122866175
iteration 4, error = 0.168951089180319
iteration 5, error = 0.005209730313222213
iteration 6, error = 4.3632751705775364e-06
iteration 7, error = 3.0460818773540415e-12
iteration 8, error = 0.0

Result = [1.49744442 1.49744442 1.49744442] 

CPU times: user 839 ms, sys: 8.37 ms, total: 848 ms
Wall time: 235 ms
----------------------------------------------------------------


We can find that Newton's method may fail for some starting values.

Sometimes it may take a few initial guesses to achieve convergence.

Substitute the result back to the formula to check our result

In [19]:
e(p, A, b, c)

Array([0., 0., 0.], dtype=float64)

We can see the result is very accurate.

```{solution-end}
```